In [1]:
# Importing modules
import pandas as pd
import os

train = pd.read_csv('./data/train.csv')
train = train.drop(columns=["Popularity"])
test = pd.read_csv('./data/test.csv')
df = train.append(test, ignore_index = True)
df.shape

(39490, 2)

In [2]:
import numpy as np
import pandas as pd
import os, itertools, csv
from bs4 import BeautifulSoup
import re
from datetime import *
import time

def get_article(text: str) -> str:    
    soup = BeautifulSoup(text, "html.parser")
    article = soup.find('section',{'class':'article-content'})
    return article.get_text()

res = []

for i in range(df.shape[0]):
    res.append(get_article(df['Page content'][i]))
df['article'] = res

In [3]:
data = df.article.values.tolist()

data

[' There may be killer asteroids headed for Earth, and NASA has decided to do something about it. The space agency announced a new "Grand Challenge" on June 18 to find all dangerous space rocks and figure out how to stop them from destroying our planet. The new mission builds on projects already underway at NASA, including a plan to capture an asteroid, pull it in toward the moon and send astronauts to visit it. As part of the Grand Challenge, the agency issued a "request for information" today — aiming to solicit ideas from industry, academia and the public on how to improve the asteroid mission plan. "We\'re asking for you to think about concepts and different approaches for what we\'ve described here," William Gerstenmaier, NASA\'s associate administrator for human explorations and operations, said yesterday during a NASA event announcing the initiative. "We want you to think about other ways of enhancing this to get the most out of it." SEE ALSO: How It Works: NASA Asteroid-Capture

In [6]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['there', 'may', 'be', 'killer', 'asteroids', 'headed', 'for', 'earth', 'and', 'nasa', 'has', 'decided', 'to', 'do', 'something', 'about', 'it', 'the', 'space', 'agency', 'announced', 'new', 'grand', 'challenge', 'on', 'june', 'to', 'find', 'all', 'dangerous', 'space', 'rocks', 'and', 'figure', 'out', 'how', 'to', 'stop', 'them', 'from', 'destroying', 'our', 'planet', 'the', 'new', 'mission', 'builds', 'on', 'projects', 'already', 'underway', 'at', 'nasa', 'including', 'plan', 'to', 'capture', 'an', 'asteroid', 'pull', 'it', 'in', 'toward', 'the', 'moon', 'and', 'send', 'astronauts', 'to', 'visit', 'it', 'as', 'part', 'of', 'the', 'grand', 'challenge', 'the', 'agency', 'issued', 'request', 'for', 'information', 'today', 'aiming', 'to', 'solicit', 'ideas', 'from', 'industry', 'academia', 'and', 'the', 'public', 'on', 'how', 'to', 'improve', 'the', 'asteroid', 'mission', 'plan', 'we', 're', 'asking', 'for', 'you', 'to', 'think', 'about', 'concepts', 'and', 'different', 'approaches', 'fo

In [58]:
n_article_token = []

for i in range(len(data_words)):
    n_article_token.append(len(data_words[i]))

n_article_token[0]

572

In [62]:
df["n_article_token"] = n_article_token

df.head()

,Id,Page content,article,LDA_0,LDA_1,LDA_2,LDA_3,LDA_4,n_article_token
0,0,"<html><head><div class=""article-info""> <span c...",There may be killer asteroids headed for Eart...,0.203237,0.000879,0.000878,0.794140,0.000866,572
1,1,"<html><head><div class=""article-info""><span cl...",Google took a stand of sorts against patent-l...,0.565165,0.001439,0.001433,0.019383,0.412581,299
2,2,"<html><head><div class=""article-info""><span cl...",You've spend countless hours training to be a...,0.000442,0.000445,0.998229,0.000441,0.000444,1059
3,3,"<html><head><div class=""article-info""><span cl...",Tired of the same old sports fails and ne...,0.001476,0.903604,0.091965,0.001469,0.001485,247
4,4,"<html><head><div class=""article-info""><span cl...","At 6-foot-5 and 298 pounds, All-Pro NFL star ...",0.000455,0.953835,0.044792,0.000460,0.000458,1311


In [63]:
unique_article_token = []

for i in range(len(data_words)):
    unique_article_token.append(len(set(data_words[i])))

unique_article_token[0]

286

In [64]:
df["unique_article_token"] = unique_article_token

df.head()

,Id,Page content,article,LDA_0,LDA_1,LDA_2,LDA_3,LDA_4,n_article_token,unique_article_token
0,0,"<html><head><div class=""article-info""> <span c...",There may be killer asteroids headed for Eart...,0.203237,0.000879,0.000878,0.794140,0.000866,572,286
1,1,"<html><head><div class=""article-info""><span cl...",Google took a stand of sorts against patent-l...,0.565165,0.001439,0.001433,0.019383,0.412581,299,152
2,2,"<html><head><div class=""article-info""><span cl...",You've spend countless hours training to be a...,0.000442,0.000445,0.998229,0.000441,0.000444,1059,539
3,3,"<html><head><div class=""article-info""><span cl...",Tired of the same old sports fails and ne...,0.001476,0.903604,0.091965,0.001469,0.001485,247,180
4,4,"<html><head><div class=""article-info""><span cl...","At 6-foot-5 and 298 pounds, All-Pro NFL star ...",0.000455,0.953835,0.044792,0.000460,0.000458,1311,664


In [10]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['killer asteroid head earth decide space agency announce new grand challenge find dangerous space rock figure how stop destroy planet new mission build project already underway include plan capture asteroid pull moon send astronaut visit part grand challenge agency issue request information today aim solicit idea industry academia public how improve asteroid mission plan re ask think concept different approach describe here administrator human exploration operation say yesterday event announce initiative want think other way enhance most see also how work request information also seek idea detect mitigating asteroid threat due mission design provide first deep space mission astronaut fly space launch system rocket orion space capsule development come fire lawmaker prefer return moon draft authorization bill subcommittee currently debate cancel mission steer agency other project bill discuss hear see also how work capture mission official defend asteroid mission today say confident win

In [34]:
# vectorizer = CountVectorizer(analyzer='word',       
#                              min_df=10,                        # minimum reqd occurences of a word 
#                              stop_words='english',             # remove stop words
#                              lowercase=True,                   # convert all words to lowercase
#                              token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
#                              # max_features=50000,             # max number of uniq words
#                             )

# data_vectorized = vectorizer.fit_transform(data_lemmatized)

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(data_lemmatized)

In [35]:
doc_term_matrix

<39490x47232 sparse matrix of type '<class 'numpy.int64'>'
	with 5634252 stored elements in Compressed Sparse Row format>

In [36]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [37]:
lda_output = LDA.fit_transform(doc_term_matrix)

In [42]:
res = lda_output.copy()

In [46]:
res = np.transpose(res)

In [50]:
res = res.tolist()

In [54]:
df["LDA_0"] = res[0]
df["LDA_1"] = res[1]
df["LDA_2"] = res[2]
df["LDA_3"] = res[3]
df["LDA_4"] = res[4]

df.head()

,Id,Page content,article,LDA_0,LDA_1,LDA_2,LDA_3,LDA_4
0,0,"<html><head><div class=""article-info""> <span c...",There may be killer asteroids headed for Eart...,0.203237,0.000879,0.000878,0.794140,0.000866
1,1,"<html><head><div class=""article-info""><span cl...",Google took a stand of sorts against patent-l...,0.565165,0.001439,0.001433,0.019383,0.412581
2,2,"<html><head><div class=""article-info""><span cl...",You've spend countless hours training to be a...,0.000442,0.000445,0.998229,0.000441,0.000444
3,3,"<html><head><div class=""article-info""><span cl...",Tired of the same old sports fails and ne...,0.001476,0.903604,0.091965,0.001469,0.001485
4,4,"<html><head><div class=""article-info""><span cl...","At 6-foot-5 and 298 pounds, All-Pro NFL star ...",0.000455,0.953835,0.044792,0.000460,0.000458


In [65]:
tmp = df.drop(columns=["Page content", "article"])
tmp.to_csv("./data/lda.csv")

In [12]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  0.7071526078005834 %


In [28]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components =10,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',
                                      learning_decay=0.9,
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = 4,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.9,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=4,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [29]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -65872455.02484529
Perplexity:  2144.3782364586286
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.9,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': 4,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [30]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Doc0,0,0,0,0.36,0,0,0.62,0,0,0.02,6
Doc1,0.57,0,0.02,0.37,0,0.03,0,0,0,0,0
Doc2,0,0,0,0.01,0,0,0,0,0.03,0.96,9
Doc3,0,0.76,0,0,0,0,0,0,0.17,0.06,1
Doc4,0,0,0,0,0,0,0,0,0.03,0.97,9
Doc5,0.28,0,0.05,0.64,0,0,0,0,0.02,0,3
Doc6,0,0,0.26,0,0,0,0,0.07,0.65,0,8
Doc7,0.08,0,0,0,0.47,0.13,0,0,0.31,0,4
Doc8,0.51,0.11,0.17,0,0,0,0.12,0.06,0.01,0,0
Doc9,0.9,0,0,0,0,0,0.04,0.06,0,0,0


In [31]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,0,7272
1,8,5888
2,3,5453
3,2,4753
4,7,3756
5,4,3309
6,5,3272
7,6,2533
8,9,1821
9,1,1433


In [32]:
df_document_topic.shape

(39490, 11)

In [33]:
df_document_topic.to_csv("./data/topic.csv")